In [14]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm

In [15]:
x_train = []
x_test = []
y_train = []

df_train = pd.read_csv('/courses/EDS232/planet/data/train_v2.csv')

flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values, miniters=1000):
    img = cv2.imread('/courses/EDS232/planet/data/train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train.append(cv2.resize(img, (32, 32)))
    y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

print(x_train.shape)
print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)
print(y_valid)
print(p_valid)
print(fbeta_score(y_valid, np.array(p_valid) > 0.2, beta=2, average='samples'))

100%|██████████| 40479/40479 [00:36<00:00, 1096.75it/s]


(40479, 32, 32, 3)
(40479, 17)


2022-03-07 21:14:13.508262: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 430080000 exceeds 10% of free system memory.


Epoch 1/4
274/274 [==============================] - 9s 31ms/step - loss: 0.2473 - accuracy: 0.4445 - val_loss: 0.1942 - val_accuracy: 0.4625
Epoch 2/4
274/274 [==============================] - 8s 29ms/step - loss: 0.2020 - accuracy: 0.4612 - val_loss: 0.1817 - val_accuracy: 0.4671
Epoch 3/4
274/274 [==============================] - 8s 28ms/step - loss: 0.1877 - accuracy: 0.4617 - val_loss: 0.1736 - val_accuracy: 0.4698
Epoch 4/4
274/274 [==============================] - 7s 27ms/step - loss: 0.1787 - accuracy: 0.4541 - val_loss: 0.1641 - val_accuracy: 0.4723
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 1 0 0]]
[[2.63124585e-01 1.78785324e-01 1.03116930e-02 ... 2.71758437e-01
  3.28004360e-03 7.94133544e-03]
 [2.77134955e-01 2.48233587e-01 9.60856676e-04 ... 2.30683118e-01
  3.40893865e-03 1.30205452e-02]
 [3.90260041e-01 1.25077724e-01 1.40610337e-03 ... 6.90042198e-01
  2.62141228e-04 8.71717930e-04]
 ...
 [3.091179